In [11]:
import sys
sys.path.append("/root/HSE_diploma/")
sys.path.append("/root/HSE_diploma/evaluator/")

In [39]:
sys.path

['/root/HSE_diploma/ipynb',
 '/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages',
 '/opt/conda/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg',
 '/root/HSE_diploma',
 '/root/HSE_diploma/evaluator/',
 '/root/HSE_diploma/evaluator/']

In [12]:
import os
import json

import transformers
from peft import PeftModel
from transformers import LlamaForCausalLM as LLaMAForCausalLM
from transformers import LlamaTokenizer as LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model
from datasets import load_dataset
from EvaluateTestSet import EvaluateTestSet
from transformers.integrations import TensorBoardCallback
from transformers import GenerationConfig


In [13]:

def init_lora_model_and_tokenizer(default_model,
                             LORA_R,
                             LORA_ALPHA,
                             LORA_DROPOUT
                            ):


    """
        
    """
    model = LLaMAForCausalLM.from_pretrained(
    default_model,
    load_in_8bit=True,
    device_map="auto",
    )
    tokenizer = LLaMATokenizer.from_pretrained(
        default_model, add_eos_token=True
    )

    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, config)

    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token

    return model, tokenizer



class MyCustomCallback(TensorBoardCallback):
    #log_bleu_steps_factor = 5
    bleu_generation_max_new_tokens = 30
    bleu_fn_test_data = "temp/t2c_answers.json"
    bleu_fn_etalon = "temp/answers.json"
    log_step = 0
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        super().on_log(args, state, control, logs=logs, **kwargs)
        #print("kwargs", len(kwargs), kwargs.keys())
        if self.tb_writer is not None:
            #print(state)
            #print(state.global_step)
            #print(self.log_step)
            if (self.log_step % self.log_bleu_steps_factor ==0):
                model = kwargs['model']
                tokenizer = kwargs['tokenizer']
                
                model.eval()
                assert not model.training
                generation_config = GenerationConfig(max_new_tokens = self.bleu_generation_max_new_tokens,
                                                     # min_new_tokens = 5,
                                                     temperature = 1.0
                                                    )
                print("generation_config:", generation_config)
                evaluator = EvaluateTestSet(generation_config = generation_config,
                                            fn_test_data = self.bleu_fn_test_data,
                                            fn_etalon = self.bleu_fn_etalon,
                                            batch_size = 1
                                       )

                metric_res = evaluator.evaluate(model=model, 
                                                tokenizer=tokenizer,
                                               )
                model.train()
                assert model.training
                print(metric_res)
                for key, val in metric_res.items():
                    #add "custom/something"
                    self.tb_writer.add_scalar(key, val, state.global_step)
                self.tb_writer.flush()
            self.log_step += 1

In [16]:
!mkdir /root/experiments/
!mkdir /root/results/

In [17]:
CONFIG_PATH = "/root/HSE_diploma/experiments_configs/"
EXPERIMENTS_PATH = "/root/experiments/"
experiment_name = "t2c_concode_220428_v34"
# t2c_concode_220428_v18.json

In [18]:
current_config_path = os.path.join(CONFIG_PATH, experiment_name + "_config.json")
experiment_config = json.load(open(current_config_path, "r"))

assert experiment_config['experiment_name'] == experiment_name

In [19]:
experiment_config['resume_from_checkpoint'] = False

In [21]:
assert experiment_config['resume_from_checkpoint'] == False

In [22]:
assert experiment_config['experiment_name'] == experiment_name

In [23]:
current_experiment_path = os.path.join(EXPERIMENTS_PATH, experiment_name)

In [25]:
print("current_experiment_path", current_experiment_path)

current_experiment_path /root/experiments/t2c_concode_220428_v34


In [26]:
!mkdir {current_experiment_path}

mkdir: cannot create directory ‘/root/experiments/t2c_concode_220428_v34’: File exists


In [27]:
json.dump(experiment_config, open(current_experiment_path + \
                                  "/experiment_config.json", 
                                  "w+"
                                 )
         )

In [28]:
setattr(MyCustomCallback, "log_bleu_steps_factor", experiment_config['log_bleu_steps_factor'])
MyCustomCallback.log_bleu_steps_factor

50

In [35]:
model, tokenizer = init_lora_model_and_tokenizer(default_model = experiment_config["default_model"],
                                                 LORA_R = experiment_config["LORA_R"],
                                                 LORA_ALPHA = experiment_config["LORA_ALPHA"],
                                                 LORA_DROPOUT = experiment_config["LORA_DROPOUT"]
                                                )


data = load_dataset("json", 
                    data_files = {"train": experiment_config["fn_train_dataset"],
                                  "eval":  experiment_config["fn_eval_dataset"]
                                 }
                   )

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-02cbc528b265cbf4/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
def verbose_model_tokenizer(model, tokenizer):
    print("tokenizer.pad_token_id", tokenizer.pad_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)
    print("tokenizer.bos_token_id", tokenizer.bos_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)

    print("model.config.pad_token_id", model.config.pad_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    print("model.config.bos_token_id", model.config.bos_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    model.print_trainable_parameters()
    
verbose_model_tokenizer(model, tokenizer)

tokenizer.pad_token_id 0
tokenizer.eos_token_id 2
tokenizer.bos_token_id 1
tokenizer.eos_token_id 2
model.config.pad_token_id 0
model.config.eos_token_id 2
model.config.bos_token_id 1
model.config.eos_token_id 2
trainable params: 8388608 || all params: 6746804224 || trainable%: 0.12433454005023165


In [37]:
experiment_config

{'experiment_name': 't2c_concode_220428_v34',
 'fn_train_dataset': '/root/data/t2c_train.json',
 'fn_eval_dataset': '/root/data/t2c_answers.json',
 'default_model': 'yahma/llama-7b-hf',
 'MICRO_BATCH_SIZE': 2,
 'BATCH_SIZE': 10,
 'EPOCHS': 2,
 'LEARNING_RATE': 0.0002,
 'CUTOFF_LEN': 256,
 'LORA_R': 16,
 'LORA_ALPHA': 16,
 'LORA_DROPOUT': 0.05,
 'warmup_steps': 200,
 'fp16': True,
 'logging_steps': 10,
 'eval_steps': 100,
 'evaluation_strategy': 'steps',
 'save_total_limit': 1,
 'save_strategy': 'steps',
 'save_steps': 500,
 'seed': 42,
 'logging_strategy': 'steps',
 'report_to': 'tensorboard',
 'mlm': False,
 'truncation': True,
 'padding': 'max_length',
 'config_use_cache': False,
 'resume_from_checkpoint': False,
 'bleu_batch_size': 5,
 'GRADIENT_ACCUMULATION_STEPS': 5,
 'log_bleu_steps_factor': 50,
 'load_best_model_at_end': False}

In [38]:
from prompter import Prompter
prompter = Prompter()

def generate_prompt(data_point):
    if "input" in data_point and data_point["input"]:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        input = data_point["input"],
                                        label = data_point["output"]
                                       )
    else:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        #input = None,
                                        label = data_point["output"]
                                       )

/root/HSE_diploma/prompter/templates/


In [40]:
experiment_config["truncation"],\
experiment_config["CUTOFF_LEN"],\
experiment_config["padding"]

(True, 256, 'max_length')

In [ ]:
# def generate_prompt(data_point):
#     # sorry about the formatting disaster gotta move fast
#     if data_point["input"]:
#         return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
# ### Instruction:
# {data_point["instruction"]}
# ### Input:
# {data_point["input"]}
# ### Response:
# {data_point["output"]}"""
#     else:
#         return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
# ### Instruction:
# {data_point["instruction"]}
# ### Response:
# {data_point["output"]}"""


data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=experiment_config["truncation"],
        max_length=experiment_config["CUTOFF_LEN"],
        padding=experiment_config["padding"]
    )
)

trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=data["train"],
    eval_dataset=data['eval'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=experiment_config["MICRO_BATCH_SIZE"],
        gradient_accumulation_steps=experiment_config["GRADIENT_ACCUMULATION_STEPS"],
        warmup_steps=experiment_config["warmup_steps"],
        num_train_epochs=experiment_config["EPOCHS"],
        learning_rate=experiment_config["LEARNING_RATE"],
        fp16=experiment_config["fp16"],
        logging_steps=experiment_config["logging_steps"],        
        evaluation_strategy = experiment_config['evaluation_strategy'],
        eval_steps=experiment_config["eval_steps"],
        output_dir=current_experiment_path,#"lora-alpaca",
        save_total_limit=experiment_config["save_total_limit"],
        save_strategy = experiment_config["save_strategy"],
        
        save_steps = experiment_config["save_steps"],
        seed=experiment_config["seed"],
        logging_dir=current_experiment_path,
        logging_strategy=experiment_config["logging_strategy"],
        report_to=experiment_config["report_to"],
        load_best_model_at_end = experiment_config["load_best_model_at_end"]
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, 
                                                               mlm=experiment_config["mlm"]
                                                              ),
    callbacks = [MyCustomCallback]
)
model.config.use_cache = experiment_config["config_use_cache"]
# print(len(trainer.optimizer.state['found_inf_per_device']))


trainer.train(resume_from_checkpoint=experiment_config["resume_from_checkpoint"])

model.save_pretrained(current_experiment_path)

In [ ]:
!df -h .

In [ ]:
!ls {current_experiment_path}